In [27]:
import pandas as pd
import os
import datetime as dt

In [28]:
class db:
    def __init__(self):
        self.transactions = db.transation_init()
        self.cc = pd.read_csv(r'db\country_codes.csv',index_col=0)
        self.customers = pd.read_csv(r'db\customers.csv',index_col=0)
        self.prod_info = pd.read_csv(r'db\prod_cat_info.csv')


    @staticmethod
    def transation_init():
        transactions = pd.DataFrame()
        print(os.getcwd())
        src = r'db/transactions/'
        for filename in os.listdir(src):
            transactions = pd.concat([transactions, pd.read_csv(os.path.join(src,filename))])

        def convert_dates(x):
            try:
                return dt.datetime.strptime(x,'%d-%m-%Y')
            except:
                return dt.datetime.strptime(x,'%d/%m/%Y')

        transactions['tran_date'] = transactions['tran_date'].apply(lambda x: convert_dates(x))

        return transactions
    
    def merge(self):
        df = self.transactions.join(self.prod_info.drop_duplicates(subset=['prod_cat_code'])
        .set_index('prod_cat_code')['prod_cat'],on='prod_cat_code',how='left')

        df = df.join(self.prod_info.drop_duplicates(subset=['prod_sub_cat_code'])
        .set_index('prod_sub_cat_code')['prod_subcat'],on='prod_subcat_code',how='left')

        df = df.join(self.customers.join(self.cc,on='country_code')
        .set_index('customer_Id'),on='cust_id')

        self.merged = df




df = db()
df.merge()

c:\Users\marszc02\OneDrive - Lantmannen Ek For\Documents\ML\Unit11.3


In [29]:
df.transactions.head()

,Unnamed: 0,transaction_id,cust_id,tran_date,prod_subcat_code,prod_cat_code,Qty,Rate,Tax,total_amt,Store_type
0,16062,40303022895,272142,2016-12-31,1,2,5,537,281.925,2966.925,Flagship store
1,16063,47557596721,273764,2016-12-31,8,3,-1,-1037,108.885,-1145.885,Flagship store
2,16064,28966519600,273899,2016-12-31,5,3,1,308,32.340,340.340,e-Shop
3,16065,18110335043,270491,2016-12-31,4,2,1,1343,141.015,1484.015,TeleShop
4,16066,35866122984,269792,2016-12-31,1,2,1,1209,126.945,1335.945,Flagship store


In [30]:
#df.transactions.set_index('tran_date')
df.transactions['DoW'] = df.transactions['tran_date'].dt.weekday.map({0: 'Poniedziałek',
                                                              1: 'Wtorek',
                                                              2: 'Środa',
                                                              3: 'Czwartek',
                                                              4: 'Piątek',
                                                              5: 'Sobota',
                                                              6: 'Niedziela'})


In [31]:
import plotly.graph_objects as go
import plotly.offline as pyo
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
#init_notebook_mode(connected=True)

fig = go.Figure()

In [32]:
df.transactions.groupby(by=['Store_type'])['total_amt'].sum().reset_index()

,Store_type,total_amt
0,Flagship store,9.715688e+06
1,MBR,9.674486e+06
2,TeleShop,9.364781e+06
3,e-Shop,1.982482e+07


In [33]:
fig = go.Figure()
fig.add_trace(go.Bar(x=df.transactions['Store_type'].unique(), y=df.transactions.groupby(by=['Store_type'])['total_amt'].sum()))
iplot(fig)